In [4]:
import joblib
import datetime
import numpy as np
from typing import Dict
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score, auc
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier

In [5]:
def _calcular_auc_por_clase(targets_reales:np.ndarray, targets_preds:np.ndarray) -> Dict[int, float]:
    """
    Computa la curva ROC y AUC para cada clase.
    :param targets_reales: Un vector de targets reales representados en 1-hot encoding.
    :param targets_preds: Un vector de targets predichos representados en 1-hot encoding.
    :return: Un diccionario de indice de categoria -> AUC de esa categoria
    """
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_clases = targets_preds.shape[1]
    for i in range(n_clases):
        fpr[i], tpr[i], _ = roc_curve(targets_reales[:, i], targets_preds[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    return roc_auc


def calcular_e_imprimir_auc(clasificador, X_train_selected, train_targets_binarios_por_clase, X_test_selected, test_targets_binarios_por_clase):
    """
    Calcular e imprime el AUC para cada categoria, utilizando el clasificador y los folds de entrenamiento y test.
    :param clasificador: Un clasificador de scikit-learn.
    :param X_train_selected: Fold de entrenamiento
    :param train_targets_binarios_por_clase: Categorias del fold de entrenamiento, en 1-hot encoding.
    :param X_test_selected: Fold de test
    :param test_targets_binarios_por_clase: Categorias del fold de test, en 1-hot encoding.
    """
    # entrenar 1 clasificador por categoria usando "one vs. rest", usamos esto para calcular AUC
    classificador_por_clase = OneVsRestClassifier(clasificador)
    # targets_preds_por_clase es una matriz donde cada fila es un vector, y cada columna es el score del clasifcador para cada categoria para la fila correspondiente de X_test_selected
    targets_preds_por_clase = classificador_por_clase.fit(X_train_selected, train_targets_binarios_por_clase).predict(X_test_selected)
    for idx_clase, valor_auc in _calcular_auc_por_clase(test_targets_binarios_por_clase, targets_preds_por_clase).items():
        print("\tAUC para la clase #{} ({}) = {}".format(idx_clase, idx_a_clase[idx_clase], valor_auc))

def pesos_de_features(score_fn, X_train, y_train) -> np.ndarray:
    scores =  np.empty((X_train.shape[1]),dtype=float)
    for i in range(0,X_train.shape[1]):
        scores[i] = score_fn(X_train[:,i],y_train)[0]
    return scores

def imprimir_features_con_pesos(score_fn, X_train, y_train, nombres_features, top_n=-1):
    """
    Esta funcion evalua que tan bien cada columna de un dataset sirve para clasificar ese dataset.
    :param score_fn: una funcion que pueda tomar una columna de feature y la columna de categoria, y calcular un score que mida que tan bien esa columna predice las categorias. Puede ser cualquier funcion dentro de sklearn.feature_selection como chi2, mutual_info_classif, o relief (si agregan relief con pip install sklearn-relief)
    :train_target_fold: una matriz con columnas a evaluar, excluyend la columna de categoria de cada fila.
    :y_train: un arreglo con el valor  categoria de  cada fila en :train_target_fold.
    :nombre_features: Los nombres de c/columna en train_target_fold.
    :top_n: cuantos de los mejores scores imprimir. -1 imprime todos.
    """
    pesos_features = pesos_de_features(score_fn, X_train, y_train)
    # conseguir los indices que ordenarian a "pesos". Como argsort solo ordena en orden ascendente, damos vuelta el arreglo
    indice_orden_desc_pesos = np.argsort(pesos_features)[::-1]
    if top_n == -1:
        top_n = X_train.shape[1]
    for i in range(0,top_n):
        print(nombres_features[indice_orden_desc_pesos[i]],'\t',pesos_features[indice_orden_desc_pesos[i]])


def nombres_features_seleccionadas(selector_features, nombres_features):
    """
    Esta funcion retorna los nombres de las columnas seleccionadas como mejores por selector_features.
    :param  selector_features: Una funcion de sklearn que puede evaluar los scores de columna y seleccionar las mejores. Puede ser SelectKBest, GenericUnivariateSelect o SelectPercentile.
    :param  nombres_features: Una lista de nombres de columnas. El orden de las columnas tiene que ser el mismo que el de la matriz con la que se evaluo a selector_features.
    :return new_features: Una lista de nombres de features que se corresponde con las seleccionadas por selector_features.
    """
    cols = selector_features.get_support()
    new_features = []
    for selected, feature in zip(cols, nombres_features):
        if selected:
            new_features.append(feature)
    return new_features

### Leer Dataset

In [6]:
vectores = joblib.load("vectores.joblib")
nombres_targets = joblib.load("targets.joblib")
nombres_features = joblib.load("features.joblib")

### Convertir Categorias

Pasar categorias a numeros (1ra categoria = 0, 2da categoria = 1, etc).

Transformar los targets en N columnas, 1 por cada categoria, donde la categoria correcta tiene un 1 y todas las demas columnas en esa fila tienen 0.

Dado que AUC se calcula sobre 2 categorias, Usamos esto luego para calcular 1 AUC por cada categoria

In [7]:
label_encoder = LabelEncoder()

targets = label_encoder.fit_transform(nombres_targets['section'])

idx_a_clase = label_encoder.classes_

n_categorias = len(idx_a_clase)

targets_binarios_por_clase = label_binarize(targets, classes=range(0, n_categorias))

### Separacion en Train/Test

In [8]:
train_index = nombres_targets['date']<datetime.datetime(2021,8,1)
X_train = vectores[train_index]
y_train = targets[train_index]

test_index = nombres_targets['date']>=datetime.datetime(2021,8,1)
X_test = vectores[test_index]
y_test = targets[test_index]

### Entreno el Modelo

In [9]:
clasificador = GradientBoostingClassifier(n_estimators=500, random_state=1234)

MAX_FEATURES=2000

# Seleccionar Features
selector_features = SelectKBest(score_func=chi2, k=MAX_FEATURES)
selector_features.fit(X_train, y_train)
X_train_selected = selector_features.transform(X_train)
X_test_selected = selector_features.transform(X_test)
selector_features.get_support()

preds_fold = clasificador.fit(X_train_selected, y_train).predict(X_test_selected)

### Resultados

In [10]:
print("Instancias Train = {}\nInstancias Test = {}".format(X_train.shape[0], X_test_selected.shape[0]))

Instancias Train = 8681
Instancias Test = 1219


#### Features Seleccionadas

In [11]:
print(nombres_features_seleccionadas(selector_features, nombres_features))

['abastecimiento', 'abdo', 'abdo benitez', 'abril dni', 'abuso', 'abuso sexual', 'acceder', 'acciones', 'acreedores', 'actas', 'actividad', 'actividad economica', 'activos', 'actos', 'actualizacion', 'acuerdo', 'acuerdo precios', 'acuerdos', 'acumulado', 'acusacion', 'acusaciones', 'acusado', 'acuso', 'ademas dia', 'adicional', 'administracion federal', 'adolescentes', 'adquisitivo', 'advirtio', 'afganistan', 'afip', 'afirmo', 'agencia', 'agentes', 'agosto', 'agravado', 'agregado', 'agricultura', 'agro', 'agroindustria', 'agroindustrial', 'ahora', 'ahorro', 'aire', 'aires', 'aires alrededores', 'aires casos', 'aires pronostico', 'aires se', 'aisladas', 'ajuste', 'al candidato', 'al capitolio', 'al ciento', 'al consumidor', 'al dolar', 'al exterior', 'al gobierno', 'al hospital', 'al lugar', 'al menos', 'al mercado', 'al precio', 'al presidente', 'al pueblo', 'al sector', 'al socialismo', 'alberto', 'alberto fernandez', 'alcalde', 'alcaldes', 'alcanzados', 'alemania', 'alerta', 'algo nu

#### Accuracy

In [12]:
accuracy_score(y_test, preds_fold)

0.911402789171452

#### Matriz de Confusion
Nota: Filas = Real - Columnas = Prediccion

In [14]:
calcular_e_imprimir_auc(
    clasificador, 
    X_train_selected, 
    targets_binarios_por_clase[train_index], 
    X_test_selected, 
    targets_binarios_por_clase[test_index]
)

confusion_matrix(y_test, preds_fold)

	AUC para la clase #0 (economy) = 0.9535565683939669
	AUC para la clase #1 (society) = 0.892910457815742
	AUC para la clase #2 (world) = 0.9310504359201385


array([[312,  16,   0],
       [ 21, 558,  50],
       [  1,  20, 241]])